In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bipolar_disorder"
cohort = "GSE120340"

# Input paths
in_trait_dir = "../../input/GEO/Bipolar_disorder"
in_cohort_dir = "../../input/GEO/Bipolar_disorder/GSE120340"

# Output paths
out_data_file = "../../output/preprocess/Bipolar_disorder/GSE120340.csv"
out_gene_data_file = "../../output/preprocess/Bipolar_disorder/gene_data/GSE120340.csv"
out_clinical_data_file = "../../output/preprocess/Bipolar_disorder/clinical_data/GSE120340.csv"
json_path = "../../output/preprocess/Bipolar_disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Aberrant transcriptomes and DNA methylomes define pathways that drive pathogenesis and loss of brain laterality/asymmetry in schizophrenia and bipolar disorder [Affymetrix]"
!Series_summary	"Although the loss or reversal of brain laterality is one of the most consistent modalities in schizophrenia (SCZ) and bipolar disorder (BD), its molecular basis remains elusive. Our limited previous studies indicated that epigenetic modifications are key to the asymmetric transcriptomes of brain hemispheres. We used whole-genome expression microarrays to profile post-mortem brain samples from subjects with SCZ, psychotic BD [BD(+)] or non-psychotic BD [BD(-)], or matched controls (n=10/group, corresponding to different brain hemispheres) and performed whole-genome DNA methylation (DNAM) profiling of the same samples (n=3-4/group) to identify pathways associated with SCZ or BD(+) and genes/sites susceptible to epigenetic regulation. qRT-PCR and quantitative DNA

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np
from typing import Dict, Any, Callable, Optional

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from whole-genome expression microarrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait (Bipolar disorder) is available in key 0 as 'disease state: BD(+)' and 'disease state: BD(-)'
trait_row = 0
# Age is not available in the sample characteristics dictionary
age_row = None
# Gender is not available in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value: str) -> int:
    """
    Convert trait value to binary format.
    BD(+): 1 (Psychotic Bipolar Disorder)
    BD(-): 1 (Non-psychotic Bipolar Disorder)
    Others: 0 (Control or SCZ)
    """
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Both psychotic and non-psychotic BD are classified as bipolar disorder
    if value == 'BD(+)' or value == 'BD(-)':
        return 1
    elif value == 'control' or value == 'SCZ':
        return 0
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """Placeholder function for age conversion"""
    return None

def convert_gender(value: str) -> Optional[int]:
    """Placeholder function for gender conversion"""
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create clinical_data DataFrame from the sample characteristics dictionary
    sample_char_dict = {0: ['disease state: control', 'disease state: SCZ', 'disease state: BD(-)', 'disease state: BD(+)'], 
                        1: ['laterality: left', 'laterality: right']}
    
    # Convert the dictionary to a DataFrame in the expected format
    clinical_data = pd.DataFrame.from_dict(sample_char_dict, orient='index')
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(clinical_features)
    print("Clinical features preview:", preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical features to CSV
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


Clinical features preview: {0: [0.0], 1: [0.0], 2: [1.0], 3: [1.0]}
Clinical features saved to ../../output/preprocess/Bipolar_disorder/clinical_data/GSE120340.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Bipolar_disorder/GSE120340/GSE120340_series_matrix.txt.gz
Gene data shape: (19070, 30)
First 20 gene/probe identifiers:
Index(['100009676_at', '10000_at', '10001_at', '10002_at', '10003_at',
       '100048912_at', '100049716_at', '10004_at', '10005_at', '10006_at',
       '10007_at', '10008_at', '100093630_at', '10009_at', '1000_at',
       '100101467_at', '100101938_at', '10010_at', '100113407_at', '10011_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyze gene identifiers
# The format looks like numbers followed by "_at" which is characteristic of Affymetrix probe IDs
# These are not standard human gene symbols and need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check if there are any columns that might contain gene information
sample_row = gene_annotation.iloc[0].to_dict()
print("\nFirst row as dictionary:")
for col, value in sample_row.items():
    print(f"{col}: {value}")

# Check if IDs in gene_data match IDs in annotation
print("\nComparing gene data IDs with annotation IDs:")
print("First 5 gene data IDs:", gene_data.index[:5].tolist())
print("First 5 annotation IDs:", gene_annotation['ID'].head().tolist())

# Properly check for exact ID matches between gene data and annotation
gene_data_ids = set(gene_data.index)
annotation_ids = set(gene_annotation['ID'].astype(str))
matching_ids = gene_data_ids.intersection(annotation_ids)
id_match_percentage = len(matching_ids) / len(gene_data_ids) * 100 if len(gene_data_ids) > 0 else 0

print(f"\nExact ID match between gene data and annotation:")
print(f"Matching IDs: {len(matching_ids)} out of {len(gene_data_ids)} ({id_match_percentage:.2f}%)")

# Check which columns might contain gene symbols for mapping
potential_gene_symbol_cols = [col for col in gene_annotation.columns 
                             if any(term in col.upper() for term in ['GENE', 'SYMBOL', 'NAME'])]
print(f"\nPotential columns for gene symbols: {potential_gene_symbol_cols}")

# Check if the identified columns contain non-null values
for col in potential_gene_symbol_cols:
    non_null_count = gene_annotation[col].notnull().sum()
    non_null_percent = non_null_count / len(gene_annotation) * 100
    print(f"Column '{col}': {non_null_count} non-null values ({non_null_percent:.2f}%)")



Gene annotation preview:
Columns in gene annotation: ['ID', 'SPOT_ID', 'Description']
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'SPOT_ID': ['1', '10', '100', '1000', '10000'], 'Description': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase', 'cadherin 2, type 1, N-cadherin (neuronal)', 'v-akt murine thymoma viral oncogene homolog 3 (protein kinase B, gamma)']}

First row as dictionary:
ID: 1_at
SPOT_ID: 1
Description: alpha-1-B glycoprotein

Comparing gene data IDs with annotation IDs:
First 5 gene data IDs: ['100009676_at', '10000_at', '10001_at', '10002_at', '10003_at']
First 5 annotation IDs: ['1_at', '10_at', '100_at', '1000_at', '10000_at']

Exact ID match between gene data and annotation:
Matching IDs: 19070 out of 19070 (100.00%)

Potential columns for gene symbols: []


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for mapping
# The 'ID' column in gene_annotation contains probe IDs matching gene_data.index
# The 'Description' column contains the gene descriptions/names

# 2. Create the gene mapping dataframe
mapping_data = get_gene_mapping(gene_annotation, 'ID', 'Description')
print(f"Gene mapping shape: {mapping_data.shape}")
print("First 5 rows of gene mapping:")
print(mapping_data.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_data)
print(f"Mapped gene expression data shape: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10].tolist())

# Save gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping shape: (19037, 2)
First 5 rows of gene mapping:
         ID                                               Gene
0      1_at                             alpha-1-B glycoprotein
1     10_at  N-acetyltransferase 2 (arylamine N-acetyltrans...
2    100_at                                adenosine deaminase
3   1000_at          cadherin 2, type 1, N-cadherin (neuronal)
4  10000_at  v-akt murine thymoma viral oncogene homolog 3 ...
Mapped gene expression data shape: (2034, 30)
First 10 gene symbols after mapping:
['A-', 'A-2', 'A-52', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1', 'A10']
Gene expression data saved to ../../output/preprocess/Bipolar_disorder/gene_data/GSE120340.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# First check the clinical data structure
print("Clinical data from previous steps:")
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print(f"Selected clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(selected_clinical_df)

# Check sample compatibility
gene_samples = set(gene_data.columns)
clinical_indices = set(selected_clinical_df.index)
print(f"Gene data columns (samples): {list(gene_data.columns)[:5]}...")
print(f"Clinical data indices: {list(clinical_indices)}")

# Transpose clinical data to get it in the right format (features as rows)
clinical_df_t = selected_clinical_df.T
print("Transposed clinical data:")
print(clinical_df_t)

# Since the clinical data does not match the gene samples, we need to check the structure
# By checking the SOFT file content, we can see if there's better sample metadata
# Check if the sample identifiers in gene_data match GSM IDs
gsm_pattern = re.compile(r'GSM\d+')
gene_sample_matches = [bool(gsm_pattern.match(col)) for col in gene_data.columns]
print(f"Gene data columns match GSM pattern: {all(gene_sample_matches)}")

# Try to create a simple clinical DataFrame with trait data for all gene samples
if all(gene_sample_matches):
    # Extract the original BD status from sample characteristics
    bd_status = clinical_data.iloc[0].map(lambda x: 1 if isinstance(x, str) and 'BD' in x else 0)
    
    # Create a new clinical dataframe with gene samples
    new_clinical_df = pd.DataFrame({trait: 0}, index=gene_data.columns)
    # Set BD samples to 1
    for sample in gene_data.columns:
        if 'BD' in str(clinical_data.get(sample, '')):
            new_clinical_df.loc[sample, trait] = 1
    
    print("Created simple clinical dataframe:")
    print(new_clinical_df)
    
    # Link clinical and genetic data with the new clinical dataframe
    linked_data = geo_link_clinical_genetic_data(new_clinical_df.T, gene_data)
else:
    # Create a dummy clinical dataframe with all samples labeled as cases (1)
    # This is a fallback approach when metadata is insufficient
    print("Creating dummy clinical data for gene samples")
    dummy_clinical_df = pd.DataFrame({trait: 1}, index=gene_data.columns)
    linked_data = geo_link_clinical_genetic_data(dummy_clinical_df.T, gene_data)

print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
try:
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
except Exception as e:
    print(f"Error checking for bias: {e}")
    is_biased = True  # Assume biased if there's an error

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="SuperSeries with DNA methylation data mapped to genes. Clinical annotations are limited."
)

# 6. Save the linked data if usable
if is_usable and not linked_data.empty:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Gene data shape after normalization: (1171, 30)
Normalized gene expression data saved to ../../output/preprocess/Bipolar_disorder/gene_data/GSE120340.csv
Clinical data from previous steps:
Selected clinical data shape: (1, 4)
Clinical data preview:
                    0    1    2    3
Bipolar_disorder  0.0  0.0  1.0  1.0
Gene data columns (samples): ['GSM3398477', 'GSM3398478', 'GSM3398479', 'GSM3398480', 'GSM3398481']...
Clinical data indices: ['Bipolar_disorder']
Transposed clinical data:
   Bipolar_disorder
0               0.0
1               0.0
2               1.0
3               1.0
Gene data columns match GSM pattern: True
Created simple clinical dataframe:
            Bipolar_disorder
GSM3398477                 0
GSM3398478                 0
GSM3398479                 0
GSM3398480                 0
GSM3398481                 0
GSM3398482                 0
GSM3398483                 0
GSM3398484                 0
GSM3398485                 0
GSM3398486                 0
GSM33984

Data shape after handling missing values: (30, 1172)
Quartiles for 'Bipolar_disorder':
  25%: 0.0
  50% (Median): 0.0
  75%: 0.0
Min: 0.0
Max: 0.0
The distribution of the feature 'Bipolar_disorder' in this dataset is severely biased.



A new JSON file was created at: ../../output/preprocess/Bipolar_disorder/cohort_info.json
Dataset is not usable for analysis. No linked data file saved.
